In [ ]:
import numpy as np
from pathlib import Path
import tensorflow as tf
from tensorflow.data import Dataset as Dataset

In [ ]:
base_path = Path("/tf/data")
class_ = [['recti'], ['prost']]

for folders in class_:
    tf.keras.backend.clear_session()
    x_sag = []
    x_tra = []
    labels = []
    for folder in folders:
        for patient in (base_path/'cropped'/'test data'/folder).iterdir():
            dates = []
            for date in patient.iterdir():
                date = date.name[:8]
                if date not in dates:
                    dates.append(date)
            for date in dates:

                #Get the sagital data
                for scan in patient.glob(f'*{date}*SAG*'):
                    volume_sag = []
                    for slice in sorted(scan.iterdir()):
                        volume_sag.append(np.load(slice, allow_pickle=True))
                
                #Get data transversal data: There is either a TRA or a TSE scan per patient. Not both.
                for scan in patient.glob(f'*{date}*TRA*'):
                    volume_tra = []
                    for slice in sorted(scan.iterdir()):
                        volume_tra.append(np.load(slice, allow_pickle=True))


                for scan in patient.glob(f'*{date}*TSE*'):
                    volume_tra = []
                    for slice in sorted(scan.iterdir()):
                        volume_tra.append(np.load(slice, allow_pickle=True))


                x_sag.append(np.array(volume_sag))
                x_tra.append(np.array(volume_tra))

                if folder == 'recti':

                    labels.append(1)
                else:

                    labels.append(0)
    ds_sag = tf.data.Dataset.from_tensor_slices(x_sag)
    ds_tra = tf.data.Dataset.from_tensor_slices(x_tra)
    ds = tf.data.Dataset.zip((ds_sag, ds_tra))
    del ds_sag, ds_tra
    ds_lab = tf.data.Dataset.from_tensor_slices(labels)
    ds = tf.data.Dataset.zip((ds, ds_lab))
    del ds_lab
    if folder == 'recti':
        save_path = "/tf/data/cropped/test data/1"
    else:
        save_path = "/tf/data/cropped/test data/0"
    tf.data.Dataset.save(ds,save_path)
    print('Saved:',save_path)